In [3]:
# -*- coding: utf-8 -*-
import pandas as pd

df_unidades = pd.read_parquet("files/ESTABLECIMIENTO_SALUD_202501.parquet")
df_unidades_ssh = pd.read_parquet("files/ESTABLECIMIENTO_SALUD_202501_ssh.parquet")

df_unidades_merge = pd.merge(df_unidades, 
                             df_unidades_ssh[['NOMBRE DE LA UNIDAD', 'CLAVE DEL MUNICIPIO', 'CLAVE DE LA LOCALIDAD', 'CLUES']], 
                             on=['NOMBRE DE LA UNIDAD', 'CLAVE DEL MUNICIPIO', 'CLAVE DE LA LOCALIDAD'],  # Se usan varias columnas como clave
                             how='left', 
                             suffixes=('', '_SSH'))
cols = list(df_unidades_merge.columns)
clues_index = cols.index('CLUES')
cols.insert(clues_index + 1, cols.pop(cols.index('CLUES_SSH')))
df_unidades_merge = df_unidades_merge[cols]
df_unidades_merge.to_excel("files/ESTABLECIMIENTO_SALUD_202501_merge.xlsx", index=False)
df_unidades_ssh.to_excel("files/ESTABLECIMIENTO_SALUD_202501_ssh.xlsx", index=False)

In [1]:

import pandas as pd
# Leer el archivo Excel
df_horarios = pd.read_excel("files/ESTABLECIMIENTO_SALUD_202501.xlsx", sheet_name="HORARIOS_202501")
# Seleccionar las columnas necesarias
df_horarios = df_horarios[['CLUES', 'HORA INICIO', 'HORA FIN']]

# Eliminar filas con valores nulos en 'HORA INICIO'
df_horarios.dropna(subset=['HORA INICIO'], inplace=True)

# Concatenar los valores de 'HORA INICIO' y 'HORA FIN'
df_horarios['HORARIO'] = df_horarios.apply(lambda row: f"De {row['HORA INICIO']} a {row['HORA FIN']}", axis=1)

# Guardar el dataframe resultante en un archivo Parquet
df_horarios.to_parquet("files/ESTABLECIMIENTO_SALUD_202501_horarios.parquet")




In [4]:

municipios = df_unidades_merge['MUNICIPIO'].unique()
df_municipios = pd.DataFrame(municipios, columns=['nombre_municipio'])
df_municipios['id_municipio'] = range(1, len(df_municipios) + 1)
  

In [5]:
filtered_df = df_unidades_merge[df_unidades_merge['MUNICIPIO'] == 'ACATLAN']
filtered_df = filtered_df[['CLUES_SSH', 'JURISDICCION', 'NOMBRE DE LA UNIDAD', 'NIVEL ATENCION']]

Agrupacion de poblacion por municipio 


In [6]:
df_poblacion_mpios_total = pd.read_excel('files/Reporte Población.xlsx', skiprows=1)
columnas_borrar = ['Clave Jurisdicción Unidad', 'Nombre Jurisdicción Unidad', 'Clave Jurisdicción Loc.', 'ageb',
                   'Clave Municipio Unidad','CLUES','Clave Localidad Unidad','Nombre Localidad Unidad',
                   'Nombre Unidad','Nombre Jurisdicción Loc','Clave Municipio Loc','Nombre Municipio Loc','Clave Localidad','Nombre Localidad']
df_poblacion_mpios_total.drop(columnas_borrar, axis=1, inplace=True)
df_poblacion_mpios_total = df_poblacion_mpios_total.groupby(['Nombre Municipio Unidad']).sum().reset_index()


Piramide poblacional Municipal Plotly

In [ ]:

import pandas as pd
import plotly.graph_objects as go
municipio = "Pachuca de Soto" 
df_municipio = df_poblacion_mpios_total[df_poblacion_mpios_total['Nombre Localidad Unidad'] == municipio]

quinquenios = ['0-4 años', '5-9 años', '10-14 años', '15-19 años', '20-24 años', '25-29 años', '30-34 años', 
               '35-39 años', '40-44 años', '45-49 años', '50-54 años', '55-59 años', '60-64 años', '65-69 años', 
               '70-74 años', '75-79 años', '80-84 años', '85+ años', 'indefinido']

poblacion_masculina = [-df_municipio[f'h{q}'].values[0] for q in quinquenios]  # Valores negativos para la gráfica
poblacion_femenina = [df_municipio[f'm{q}'].values[0] for q in quinquenios]

fig = go.Figure()

fig.add_trace(go.Bar(
    y=quinquenios,
    x=poblacion_masculina,
    name='Hombres',
    orientation='h',
    marker=dict(color='blue'),
    hovertemplate='%{y}: %{text}<extra></extra>',
    text=[abs(val) for val in poblacion_masculina]
))

fig.add_trace(go.Bar(
    y=quinquenios,
    x=poblacion_femenina,
    name='Mujeres',
    orientation='h',
    marker=dict(color='pink'),
    hovertemplate='%{y}: %{x}<extra></extra>',
    text=[abs(val) for val in poblacion_masculina]
))

max_val = max(max(poblacion_femenina), abs(min(poblacion_masculina)))
tickvals = [-max_val, -max_val/2, 0, max_val/2, max_val]
ticktext = [str(int(abs(val))) for val in tickvals]

fig.update_layout(
    title=f'Pirámide Poblacional de {municipio}',
    xaxis_title='Población',
    yaxis_title='Edad',
    barmode='relative',
    bargap=0.1,
    bargroupgap=0,
    xaxis=dict(
        tickvals=tickvals,
        ticktext=ticktext
    )
)
fig.show()

Agrupacion de poblacion por unidad de salud 

In [ ]:
df_poblacion = pd.read_excel('files/Reporte Población.xlsx', skiprows=1)
columnas_borrar = ['Clave Jurisdicción Unidad', 'Nombre Jurisdicción Unidad', 'Clave Jurisdicción Loc.', 'ageb',
                   'Clave Municipio Unidad','Nombre Municipio Unidad','Clave Localidad Unidad','Nombre Localidad Unidad',
                   'Nombre Unidad','Nombre Jurisdicción Loc','Clave Municipio Loc','Nombre Municipio Loc','Clave Localidad','Nombre Localidad']
df_poblacion.drop(columnas_borrar, axis=1, inplace=True)

df_poblacion_ssh = df_poblacion[~df_poblacion['CLUES'].str.startswith('HGIMO')]#se omiten las clues que inician con HGIMO
df_poblacion_ssh_group = df_poblacion_ssh.groupby('CLUES').sum().reset_index()

Piramide poblacional por CLUES

In [ ]:
import plotly.graph_objects as go
clues = 'HGSSA001923'
df_clues = df_poblacion_ssh_group[df_poblacion_ssh_group['CLUES'] == clues]
quinquenios = ['0-4 años', '5-9 años', '10-14 años', '15-19 años', '20-24 años', '25-29 años', '30-34 años', 
               '35-39 años', '40-44 años', '45-49 años', '50-54 años', '55-59 años', '60-64 años', '65-69 años', 
               '70-74 años', '75-79 años', '80-84 años', '85+ años', 'indefinido']

poblacion_masculina = [-df_clues[f'h{q}'].values[0] for q in quinquenios]  # Valores negativos para la gráfica
poblacion_femenina = [df_clues[f'm{q}'].values[0] for q in quinquenios]

fig = go.Figure()

fig.add_trace(go.Bar(
    y=quinquenios,
    x=poblacion_masculina,
    name='Hombres',
    orientation='h',
    marker=dict(color='blue'),
    hovertemplate='%{y}: %{text}<extra></extra>',
    text=[abs(val) for val in poblacion_masculina]
))

fig.add_trace(go.Bar(
    y=quinquenios,
    x=poblacion_femenina,
    name='Mujeres',
    orientation='h',
    marker=dict(color='pink'),
    hovertemplate='%{y}: %{x}<extra></extra>',
    text=[abs(val) for val in poblacion_masculina]
))

max_val = max(max(poblacion_femenina), abs(min(poblacion_masculina)))
tickvals = [-max_val, -max_val/2, 0, max_val/2, max_val]
ticktext = [str(int(abs(val))) for val in tickvals]

fig.update_layout(
    title=f'Pirámide Poblacional de {clues}',
    xaxis_title='Población',
    yaxis_title='Edad',
    barmode='relative',
    bargap=0.1,
    bargroupgap=0,
    xaxis=dict(
        tickvals=tickvals,
        ticktext=ticktext
    )
)
fig.show()